In [1]:
import pandas as pd
import numpy as np
import os




In [2]:
folder_path = '../Data/luad_oncosg_2020'
file_name = 'data_mrna_seq_v2_rsem.txt'
file_path = folder_path + '/' + file_name
# Load the RNAseq data from the csv file
data = pd.read_csv(file_path, sep='\t', index_col=0)

# Get the column names
data.head()
# the columns correspond to the patient IDs

,Entrez_Gene_Id,A183,A185,A186,A187,A188,A190,A191,A192,A193,...,A510,A524,A522,A521,A519,A534,A530,A532,A538,A537
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,7105,0.456572,0.813514,-1.491180,0.490566,-0.038453,0.104194,2.292825,-0.366197,0.416299,...,1.146403,-1.212214,-0.251335,0.316920,0.522394,0.782396,0.001845,-1.643590,0.470657,-0.860575
TNMD,64102,-0.578293,-0.578293,0.201038,-0.578293,2.207833,0.010727,-0.578293,0.033853,-0.578293,...,0.275027,0.879621,-0.102213,3.454547,-0.578293,-0.578293,-0.578293,-0.578293,-0.139295,-0.578293
DPM1,8813,0.746824,0.776922,0.629568,0.380121,-0.979192,0.797896,0.594240,1.006989,0.337537,...,0.347499,0.519170,-0.439873,0.115883,-0.021129,0.346257,-0.716488,0.420377,0.690033,0.510026
SCYL3,57147,-0.360768,1.131497,-0.137962,0.615019,0.173132,0.656928,1.298342,-1.232742,0.136089,...,-0.785241,0.014876,-0.951885,-0.981716,-0.860428,1.179561,0.579385,-1.281803,-0.849154,-0.682188
C1orf112,55732,0.442399,0.554759,0.490482,0.879593,-0.883658,1.061832,-0.526089,-0.708989,0.960094,...,-0.337896,-0.667930,-1.116340,1.051841,-0.262754,-1.146648,-1.058235,-0.679668,-0.127222,-0.517206


In [3]:
# Reverse the log2 transformation for the RNAseq data all columns but the 0 and 1st and subtract the pseudo-count
data.iloc[:,2:] = (2**data.iloc[:,2:]-1) 

In [4]:
data.head()

,Entrez_Gene_Id,A183,A185,A186,A187,A188,A190,A191,A192,A193,...,A510,A524,A522,A521,A519,A534,A530,A532,A538,A537
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,7105,0.456572,0.757487,-0.644279,0.404996,-0.026302,0.074894,3.900147,-0.224175,0.334500,...,1.213613,-0.568394,-0.159882,0.245668,0.436337,0.719985,0.001280,-0.679941,0.385740,-0.449267
TNMD,64102,-0.578293,-0.330244,0.149525,-0.330244,3.619809,0.007463,-0.330244,0.023743,-0.330244,...,0.210017,0.839892,-0.068397,9.962823,-0.330244,-0.330244,-0.330244,-0.330244,-0.092037,-0.330244
DPM1,8813,0.746824,0.713472,0.547101,0.301451,-0.492736,0.738564,0.509677,1.009712,0.263598,...,0.272353,0.433131,-0.262801,0.083638,-0.014539,0.271258,-0.391423,0.338277,0.613321,0.424076
SCYL3,57147,-0.360768,1.190860,-0.091198,0.531579,0.127504,0.576721,1.459461,-0.574492,0.098922,...,-0.419745,0.010365,-0.483044,-0.493623,-0.449211,1.265078,0.494212,-0.588719,-0.444890,-0.376780
C1orf112,55732,0.442399,0.468924,0.404914,0.839856,-0.458009,1.087580,-0.305565,-0.388251,0.945437,...,-0.208806,-0.370591,-0.538737,1.073173,-0.166505,-0.548327,-0.519781,-0.375691,-0.084407,-0.301276


In [5]:
# ! PROBLEM! I don't have the transcript length data

expression_data = data.iloc[:,2:]
# Load transcript length data (assuming it's in a separate csv file with the same index of Hugo_symbol)
# transcript_length_data = pd.read_csv('transcript_length_data.csv', index_col=0)
# For simplicity, let's assume all transcripts are 1000 bases long
transcript_length_data = pd.DataFrame(np.full_like(expression_data, fill_value=1000), index=expression_data.index, columns=expression_data.columns)

# Convert transcript length from bases to kilobases
transcript_length_kb = transcript_length_data / 1000

# Calculate the Length Normalization Factor (LNF)
lnf_data = expression_data / transcript_length_kb

# Sum all LNF values across all genes for each patient
sum_LNF = lnf_data.sum(axis=0)

# Calculate TPM
tpm_data = (lnf_data.div(sum_LNF, axis=1)) * 1e6

# Now tpm_data contains the TPM values
# Optionally, save the transformed data to a new csv file
tpm_data.to_csv('luad_oncosg_2020_tpm.csv')